# Perform an online prediction from the deployed model
* This leverages the model we deployed before to make a prediction as soon as a request comes in.

In [ ]:
!pip install -q optimum[onnxruntime]==1.23.3 transformers==4.46.3 torch==2.2.2+cu121
import time
import requests

We need some preprocessing functions from the model

In [ ]:
from optimum.onnxruntime import ORTModelForImageClassification
from transformers import AutoFeatureExtractor

# Load and export the model to ONNX
model = ORTModelForImageClassification.from_pretrained("rh-ai-bu/wildfire01", export=True)

# Load the feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("rh-ai-bu/wildfire01")

# Create an ONNX-based pipeline
from optimum.pipelines import pipeline
onnx_pipeline = pipeline("image-classification", model=model, feature_extractor=feature_extractor, accelerator="ort")

In [ ]:
onnx_pipeline.save_pretrained("onnx_pipeline")

In [ ]:
# image = "pic2.jpg"
# Import the function from the script
from get_random_image import get_random_image

# Call the function
image = get_random_image()

# Print the selected image path
print(image)

image = "06-wildfire-sample/wildfire_types_image_detection_sample/val/fire/Smoke_from_fires/50323627386_af76996942_o.jpg"


In [ ]:
## show the image we are working with
from IPython.display import Image, display
display(Image(filename=image, width=1000))

In [ ]:
image_preprocessed = onnx_pipeline.preprocess(image)

In [ ]:
image_preprocessed["pixel_values"].shape

In [ ]:
## figure out the precise URL we need to hit:

import subprocess

def run_command(command):
    """Run an OS command and return its output, handling errors."""
    try:
        result = subprocess.check_output(command, shell=True, text=True).strip()
        return result
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while running the command '{command}': {e}")
        return None

# Commands to run
get_IS_name = "oc get inferenceservice wildfire01 -o jsonpath='{.metadata.name}'"
get_IS_URL = "oc get inferenceservice wildfire01 -o jsonpath='{.status.url}'"

# Retrieve results
deployed_model_name = run_command(get_IS_name)
infer_endpoint = run_command(get_IS_URL)

# Print the results
if deployed_model_name:
    print("Inference Service Name:", deployed_model_name)
else:
    print("Failed to retrieve Inference Service Name.")

if infer_endpoint:
    print("Inference Service URL:", infer_endpoint)
else:
    print("Failed to retrieve Inference Service URL.")


In [ ]:
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"
print(infer_url)

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "pixel_values",
                "shape": image_preprocessed["pixel_values"].shape,
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    start_time = time.time()  # Record the start time
    response = requests.post(infer_url, json=json_data, verify=True)
    end_time = time.time()  # Record the end time

    # Calculate and print the response time
    response_time = end_time - start_time
    print(f"Response time: {response_time:.6f} seconds")

    return response

In [ ]:
prediction = rest_request(image_preprocessed["pixel_values"].tolist())
prediction.json()